In [1]:
#pip install textblob
#pip install tweepy
#pip install afinn
#pip install autocorrect
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('sentiwordnet')
#pip install spacy vaderSentiment

In [8]:
# Import dependencis
import pandas as pd
import itertools  
from textblob import TextBlob
import sys, tweepy
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
import matplotlib.pyplot as plt
import csv
import ssl
import time
import warnings
warnings.filterwarnings('ignore') # Hides warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)
import nltk
from nltk.corpus import stopwords
from techniques import *
from autocorrect import Speller

In [9]:
# File paths
tweet_data_file = os.path.join("..","Resources","data","tdata.csv")
cleaned_tweet_file=os.path.join("..","Resources","data","tweetCleandata.csv")

### Prepare Data
- Social media data is unstructured -it’s raw, noisy, and needs to be cleaned before we can start working on our sentiment analysis model. 
- Preprocessing a Twitter dataset involves a series of tasks like removing all types of irrelevant information like emojis, special characters, and extra blank spaces. It can also involve making format improvements, delete duplicate tweets, or tweets that are shorter than three characters.

In [11]:
# Load tweet csv file
tweet_df= pd.read_csv(tweet_data_file)

In [12]:
# Check for null values
tweet_df.isna().sum()

Tweet               0
Matched Keywords    0
Date                0
User                0
Source              2
Tweet ID            0
Tweet URL           0
Followers           0
Friends             0
Favorite            0
dtype: int64

In [13]:
# Drop rows with null values
tweet_df=tweet_df.dropna()

In [14]:
# Check df info
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59591 entries, 0 to 59592
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tweet             59591 non-null  object 
 1   Matched Keywords  59591 non-null  object 
 2   Date              59591 non-null  object 
 3   User              59591 non-null  object 
 4   Source            59591 non-null  object 
 5   Tweet ID          59591 non-null  float64
 6   Tweet URL         59591 non-null  object 
 7   Followers         59591 non-null  int64  
 8   Friends           59591 non-null  int64  
 9   Favorite          59591 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 5.0+ MB


In [15]:
# Get value count for each keyword
tweet_df['Matched Keywords'].value_counts()

Trump    48817
Biden    10774
Name: Matched Keywords, dtype: int64

In [16]:
#Lets copy column
tweet_df['CleanedTweet'] = tweet_df['Tweet']

In [17]:
# Lets clean the tweet data

In [18]:
# 1. Remove unicode
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: removeUnicode(x))

In [19]:
tweet_df[['Tweet','CleanedTweet']]

,Tweet,CleanedTweet
0,Breaking #FoxNews Alert : Biden scraps plans t...,Breaking #FoxNews Alert : Biden scraps plans t...
1,@JRubinBlogger Isn't the doctor who gave Trump...,@JRubinBlogger Isn't the doctor who gave Trump...
2,@WindsorMann The only friends Trump has had hi...,@WindsorMann The only friends Trump has had hi...
3,Trump’s pick to manage public lands has four-d...,Trumps pick to manage public lands has four-de...
4,Because Trump is an asshole: https://t.co/Vs5r...,Because Trump is an asshole: https://t.co/Vs5r...
...,...,...
59588,This is an attempt to siphon votes from Joe Bi...,This is an attempt to siphon votes from Joe Bi...
59589,"Covid, Boris Johnson, Trump, climate crisis an...","Covid, Boris Johnson, Trump, climate crisis an..."
59590,@ddale8 Trump is a “thing”. Isn’t he? When wil...,@ddale8 Trump is a thing. Isnt he? When will h...
59591,"@JLBreezy00 Oh yea, TRUMP 2020!! GOD BLESS AM...","@JLBreezy00 Oh yea, TRUMP 2020!! GOD BLESS AM..."


In [20]:
#2. Replace URL, hashTag etc.
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceURL(x))
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceAtUser(x))
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: removeHashtagInFrontOfWord(x))

In [21]:
#3. Replace slang words and abbreviations with their equivalents
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceSlang(x))

In [22]:
#4. Replace contractions to their equivalents
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceContraction(x))

In [23]:
#5. Remove integers from text
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: removeNumbers(x))

In [24]:
#6. Remove emoticons from text   
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: removeEmoticons(x))

In [25]:
#7. Replace repetitions of exlamation marks,question marks and stop marks
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceMultiExclamationMark(x))
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceMultiQuestionMark(x))
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: replaceMultiStopMark(x))

In [26]:
tweet_df[['Tweet','CleanedTweet']]

,Tweet,CleanedTweet
0,Breaking #FoxNews Alert : Biden scraps plans t...,Breaking FoxNews Alert Biden scraps plans to ...
1,@JRubinBlogger Isn't the doctor who gave Trump...,atUser Is not the doctor who gave Trump the te...
2,@WindsorMann The only friends Trump has had hi...,atUser The only friends Trump has had his enti...
3,Trump’s pick to manage public lands has four-d...,Trumps pick to manage public lands has four-de...
4,Because Trump is an asshole: https://t.co/Vs5r...,Because Trump is an asshole url
...,...,...
59588,This is an attempt to siphon votes from Joe Bi...,This is an attempt to siphon votes from Joe Bi...
59589,"Covid, Boris Johnson, Trump, climate crisis an...","Covid, Boris Johnson, Trump, climate crisis an..."
59590,@ddale8 Trump is a “thing”. Isn’t he? When wil...,atUser Trump is a thing. Isnt he? When will he...
59591,"@JLBreezy00 Oh yea, TRUMP 2020!! GOD BLESS AM...","atUser Oh yea, TRUMP multiExclamation GOD B..."


In [27]:
def rmPunctuation(text):
    return re.sub(r'[^\w\s]', '', text) 

In [28]:
# 8: Remove punctuation
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: rmPunctuation(x))

In [29]:
# Make predTweet Column to have a minnigful sentence for predictor
tweet_df['predTweet']=tweet_df['CleanedTweet']

In [30]:
# Display results from earlier steps
tweet_df[['predTweet','CleanedTweet']]

,predTweet,CleanedTweet
0,Breaking FoxNews Alert Biden scraps plans to ...,Breaking FoxNews Alert Biden scraps plans to ...
1,atUser Is not the doctor who gave Trump the te...,atUser Is not the doctor who gave Trump the te...
2,atUser The only friends Trump has had his enti...,atUser The only friends Trump has had his enti...
3,Trumps pick to manage public lands has fourdec...,Trumps pick to manage public lands has fourdec...
4,Because Trump is an asshole url,Because Trump is an asshole url
...,...,...
59588,This is an attempt to siphon votes from Joe Bi...,This is an attempt to siphon votes from Joe Bi...
59589,Covid Boris Johnson Trump climate crisis and t...,Covid Boris Johnson Trump climate crisis and t...
59590,atUser Trump is a thing Isnt he When will he f...,atUser Trump is a thing Isnt he When will he f...
59591,atUser Oh yea TRUMP multiExclamation GOD BL...,atUser Oh yea TRUMP multiExclamation GOD BL...


In [31]:
# This is just to remove converted charactersets from a tweet that that will be used for prediction
def filterStopWords(text):
    filterwords = "multiexclamation multiquestion multistop url atuser am pm" 
    finalTokens = [] # all tokens
    tokens = text.split()
    for w in tokens:
        #print(w)
        w=w.lower()
        # 9. Remove some stopwords
        if (w not in filterwords):
            #print(w)
            final_word = w.lower()
            finalTokens.append(final_word)
    return " ".join(finalTokens)

In [32]:
# Filter stop words
tweet_df['predTweet']=tweet_df['predTweet'].apply(lambda x: filterStopWords(x))

In [33]:
# Display results from earlier steps
tweet_df[['predTweet','CleanedTweet']]

,predTweet,CleanedTweet
0,breaking foxnews alert biden scraps plans trav...,Breaking FoxNews Alert Biden scraps plans to ...
1,not the doctor who gave trump the test the sam...,atUser Is not the doctor who gave Trump the te...
2,the only friends trump has had his entire life...,atUser The only friends Trump has had his enti...
3,trumps pick manage public lands has fourdecade...,Trumps pick to manage public lands has fourdec...
4,because trump an asshole,Because Trump is an asshole url
...,...,...
59588,this an attempt siphon votes from joe biden ka...,This is an attempt to siphon votes from Joe Bi...
59589,covid boris johnson trump climate crisis and t...,Covid Boris Johnson Trump climate crisis and t...
59590,trump thing isnt he when will he finally just ...,atUser Trump is a thing Isnt he When will he f...
59591,oh yea trump god bless america and all its mil...,atUser Oh yea TRUMP multiExclamation GOD BL...


In [34]:
# Tokenizes a text to its words, removes and replaces some of them 
stoplist = stopwords.words('english')
my_stopwords = "multiexclamation multiquestion multistop gave url atuser st rd nd th am pm" # my extra stopwords
stoplist = stoplist + my_stopwords.split()
allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging
lemmatizer = WordNetLemmatizer() # set lemmatizer
stemmer = PorterStemmer() # set stemmer
spell = Speller(fast=True)

In [35]:
# Function to lower case words and remove stop words
def exCleanup(text):
    finalTokens = [] # all tokens
    # Get tokens
    tokens = nltk.word_tokenize(text)
    
    # 8. Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym
    #tokens = replaceNegations(text) 
        
    for w in tokens:
        w= w.lower()
        # 9. Remove stopwords
        if (w not in stoplist):
            # 10. lowercases all characters
            final_word = w.lower()
            finalTokens.append(final_word)

  
    return finalTokens

In [36]:
# Function for text processing - spell check, elongated words, lamatizr and stemming of data
def exSpeechTag(text):    
    finalTokens = [] # all tokens
    tokens = text #as data is alrady tokenized
    
    for w in tokens:

        if (w not in stoplist and len(w)>3):
            
            # 11. Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_
            final_word = addCapTag(w)
           
            # 12. Replaces an elongated word with its basic form, unless the word exists in the lexicon
            final_word = replaceElongated(final_word)
          
            if len(final_word)>1:
                # 13. Correction of spelling errors
                final_word = spell(final_word)
                #print(final_word)
            # 14. lemmatizes words   
            final_word = lemmatizer.lemmatize(final_word)
            # 15. Apply stemming to words
            final_word = stemmer.stem(final_word)
                          
            finalTokens.append(final_word)
    return finalTokens

In [37]:
# Tokenize tweet
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: exCleanup(x))

In [38]:
# Spell check,replace elongated, lammatize and stemming
tweet_df['CleanedTweet']=tweet_df['CleanedTweet'].apply(lambda x: exSpeechTag(x))

In [39]:
# Display results from earlier steps
tweet_df[['Tweet','CleanedTweet']]

,Tweet,CleanedTweet
0,Breaking #FoxNews Alert : Biden scraps plans t...,"[break, foxnew, alert, bien, scrap, plan, trav..."
1,@JRubinBlogger Isn't the doctor who gave Trump...,"[doctor, trump, test, doctor, tri, head, quid,..."
2,@WindsorMann The only friends Trump has had hi...,"[friend, trump, entir, life, one, bought, mone..."
3,Trump’s pick to manage public lands has four-d...,"[trump, pick, manag, public, land, fourdecad, ..."
4,Because Trump is an asshole: https://t.co/Vs5r...,"[trump, asshol]"
...,...,...
59588,This is an attempt to siphon votes from Joe Bi...,"[attempt, siphon, vote, bien, wayn, friend, ne..."
59589,"Covid, Boris Johnson, Trump, climate crisis an...","[covid, ori, johnson, trump, climat, crisi, va..."
59590,@ddale8 Trump is a “thing”. Isn’t he? When wil...,"[trump, thing, isnt, final, away, like, thing,..."
59591,"@JLBreezy00 Oh yea, TRUMP 2020!! GOD BLESS AM...","[trump, bless, america, militari, polic, everi..."


In [40]:
# Change date column format and export only required columns for further processing
tweet_df['Date'] = pd.to_datetime(tweet_df['Date'])
tweet_df['Date'] = tweet_df['Date'].dt.strftime('%m/%d/%Y')

In [41]:
#Let's clean orginal tweet for spell check, removing 
tweet_df['Tweet']=tweet_df['Tweet'].apply(lambda x: spell(x))

In [42]:
# Select only required columns
tweet_df=tweet_df[['Date','Matched Keywords','User','Source','Followers','Friends','Favorite','Tweet','predTweet','CleanedTweet']]

In [43]:
# Display results
tweet_df.head()

,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet
0,08/05/2020,Biden,YourSiteingGoog,IFTTT,84,198,0,Breaking #FoxNews Alert : Bien scraps plans to...,breaking foxnews alert biden scraps plans trav...,"[break, foxnew, alert, bien, scrap, plan, trav..."
1,08/05/2020,Trump,rbenton75nc,Twitter for Android,15,87,0,@JRubinBlogger Isn't the doctor who gave Trump...,not the doctor who gave trump the test the sam...,"[doctor, trump, test, doctor, tri, head, quid,..."
2,08/05/2020,Trump,Mrb18101,Twitter for iPhone,72,134,0,@WindsorMann The only friends Trump has had hi...,the only friends trump has had his entire life...,"[friend, trump, entir, life, one, bought, mone..."
3,08/05/2020,Trump,mermansteve,Twitter Web App,5232,5664,0,Trump’s pick to manage public lands has four-d...,trumps pick manage public lands has fourdecade...,"[trump, pick, manag, public, land, fourdecad, ..."
4,08/05/2020,Trump,biospherian,Twitter Web App,26,167,0,Because Trump is an asshole: http://t.co/Is5rk...,because trump an asshole,"[trump, asshol]"


In [44]:
# Save Clean tweet data for charting and modelling
tweet_df.to_csv(cleaned_tweet_file,index=False)